In [1]:
import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
import spacy

import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.metrics.pairwise import cosine_similarity

import random
import string

In [2]:
# Reading in the prompts and response database
qandadb = pd.read_excel('q&adatabase.xlsx')

# Changing questions column to lowercase
qandadb['questions'] = qandadb['questions'].apply(lambda x : x.lower())

# Droping all the duplicates that appear in each stage
qandadb = qandadb.drop_duplicates(subset=['stage', 'questions', 'resp'], keep='first')

In [3]:
stopwords = [
             'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 
             'ourselves', 'you', "you're", "you've", "you'll", 
             "you'd", 'your', 'yours', 'yourself', 'yourselves', 
             'he', 'him', 'his', 'himself', 'she', "she's", 'her', 
             'hers', 'herself', 'it', "it's", 'its', 'itself', 
             'they', 'them', 'their', 'theirs', 'themselves', 'what', 
             'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 
             'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 
             'being', 'have', 'has', 'had', 'having', 'do', 'does', 
             'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 
             'or', 'because', 'as', 'until', 'while', 'of', 'at', 
             'by', 'for', 'with', 'about', 'against', 'between', 
             'into', 'through', 'during', 'before', 'after', 'above', 
             'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 
             'off', 'over', 'under', 'again', 'further', 'then', 'once', 
             'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 
             'both', 'each', 'few', 'more', 'most', 'other', 'some', 
             'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 
             'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 
             'don', "don't", 'should', "should've", 'now', 'd', 'll', 
             'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', 
             "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', 
             "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 
             'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 
             'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren',
             "weren't", 'won', "won't", 'wouldn', "wouldn't", 'fuck', 'fucker',
             'lah', 'la', 'leh', 'lor', 'nah', 'ya', 'yah', 'shit', 'ass', 'asshole',
             'le', 'already', 'liao', 'liaoz', 'u', 'cheebye', 'lanjiao',
             'nabei', 'kaopei', 'knnb', 'cb', 'cheebye', 'fucked', 'fucks',
             'bitch', 'bitches', 'scumbag', 'fuckface', 'wtf', 'ffs', 'siao',
             'walao', 'waliao', 'ttyl', 'orhhhh', 'sai'
             ]

In [4]:
# Function to convert questions in each stage to one string
def stage2string(stage_name):
    string = ''   
    for x in qandadb[qandadb['stage'] == stage_name]['questions']:
        string += str(x + ' ')
    return string

In [5]:
stage_name = qandadb['stage'].unique().tolist()

for stage in stage_name:
    print(stage2string(stage))
    print('---------------------------------')

i have settled my payment with my consultant. i have made my payment with my consultant. when do i have to make payment? how do i make payment? is there any late payment fee? can i request for an extension of payment? i want to meet with my consultant. how do i pay with my skills future credits? how do i pay with my sfc? when must i submit my sfc? no, i do not need help. yes, i need help. yes, i need assistance. yes, i want to meet the mentor. how do i read the timetable? how does the attendance system work? how do we take attendance? how does the attendance work? when can i contact my mentor? how do i contact my mentor for online support? what is my mentor's email address? what is my mentor's phone number? 
---------------------------------
i am having an issue in accessing the e-content in lms. i am having difficulty in understanding the e-content. i am having difficulty in understanding the course content. is it compulsory to complete the e-learning? till which date i can complete e

i am having an issue in accessing the e-content in lms. i am having difficulty in understanding the e-content. i am having difficulty in understanding the course content. is it compulsory to complete the e-learning? till which date i can complete e-learning? what if i do not complete the e-learning on time?  i do not have time to complete my mcqs. how many times can i attempt the mcqs? what should i do? till which date i can complete mcqs? what if i do not complete the mcqs on the time?  what if i do not attempt the mcqs? is there is any marking for mcqs? where can i see my progress? what is the site to access my e-book? can i download or print the e-book? do you have a hard copy of the e-book? what is the next step? 
---------------------------------
i am having an issue in accessing the e-content in lms. i am having difficulty in understanding the e-content. i am having difficulty in understanding the course content. is it compulsory to complete the e-learning? till which date i can 

i do not have time to complete my assignment. how many times can i upload the assignments? who will score my assignment? when can i see my assignment score? till which date i can complete assignments? what if i do not complete the assignments on the time?  what if i do not attempt the assignments? where can i see my progress? what should i do if i did not score 70% for my assignment? who can i approach to reset my assignment? why i can’t log into the learn-on-demand? where do i upload the lab assignment? why can’t i see the upload button in the lms? why can’t i launch the lab? where do i get the template? what is the site to access my e-book? can i download or print the e-book? do you have a hard copy of the e-book? is it compulsory to attend the mentoring session? i am not able to attend the mentoring session. how was the attendance capture for the mentoring session? i have no issue with my assignment. can i skip the mentoring session? i have completed my assignment in advance. can i 

In [6]:
# Stage dictionary
soc_days = "i have settled my payment with my consultant. i have made my payment with my consultant. when do i have to make payment? how do i make payment? is there any late payment fee? can i request for an extension of payment? i want to meet with my consultant. how do i pay with my skills future credits? how do i pay with my sfc? when must i submit my sfc? no, i do not need help. yes, i need help. yes, i need assistance. yes, i want to meet the mentor. how do i read the timetable? how does the attendance system work? how do we take attendance? how does the attendance work? when can i contact my mentor? how do i contact my mentor for online support? what is my mentor's email address? what is my mentor's phone number?"
soc_el1 = 'i am having an issue in accessing the e-content in lms.i am having difficulty in understanding the e-content.i am having difficulty in understanding the course content.is it compulsory to complete the e-learning?till which date i can complete e-learning?what if i do not complete the e-learning on time? i do not have time to complete my mcqs.how many times can i attempt the mcqs?what should i do?till which date i can complete mcqs?what if i do not complete the mcqs on the time? what if i do not attempt the mcqs?is there is any marking for mcqs?where can i see my progress?what is the site to access my e-book?can i download or print the e-book?do you have a hard copy of the e-book?what is the next step?'
el1_el2 = 'i am having an issue in accessing the e-content in lms.i am having difficulty in understanding the e-content.i am having difficulty in understanding the course content.is it compulsory to complete the e-learning?till which date i can complete e-learning?what if i do not complete the e-learning on time? i do not have time to complete my mcqs.how many times can i attempt the mcqs?what should i do?till which date i can complete mcqs?what if i do not complete the mcqs on the time? what if i do not attempt the mcqs?is there is any marking for mcqs?where can i see my progress?what is the site to access my e-book?can i download or print the e-book?do you have a hard copy of the e-book?what is the next step?'
el2_fc1 = 'i am having an issue in accessing the e-content in lms.i am having difficulty in understanding the e-content.i am having difficulty in understanding the course content.is it compulsory to complete the e-learning?till which date i can complete e-learning?what if i do not complete the e-learning on time? i do not have time to complete my mcqs.how many times can i attempt the mcqs?what should i do?till which date i can complete mcqs?what if i do not complete the mcqs on the time? what if i do not attempt the mcqs?is there is any marking for mcqs?where can i see my progress?when is my flipped class?what should i prepare for my flipped class?what if i cannot attend the flipped class. what can i do?is there any make-up flipped class?do i have to submit my assignment during flipped class?what is the site to access my e-book?can i download or print the e-book?do you have a hard copy of the e-book?what is the next step?what should i prepare for my flipped class?what if i cannot attend the flipped class. what can i do?is there any make-up flipped class?do i have to submit my assignment during flipped class?'
fc1_a1 = 'i do not have time to complete my assignment.how many times can i upload the assignments?who will score my assignment?when can i see my assignment score?till which date i can complete assignments?what if i do not complete the assignments on the time? what if i do not attempt the assignments?where can i see my progress?what should i do if i did not score 70% for my assignment?who can i approach to reset my assignment?why i can’t log into the learn-on-demand?where do i upload the lab assignment?why can’t i see the upload button in the lms?why can’t i launch the lab?where do i get the template?what is the site to access my e-book?can i download or print the e-book?do you have a hard copy of the e-book?is it compulsory to attend the mentoring session?i am not able to attend the mentoring session.how was the attendance capture for the mentoring session?i have no issue with my assignment. can i skip the mentoring session?i have completed my assignment in advance. can i skip the mentoring session?what is the next step?when is my next flipped class?what should i prepare for my next flipped class?what if i cannot attend the next flipped class. what can i do?is there any make-up flipped class?do i have to submit my assignment during flipped class?how was the attendance capture for the mentoring session?why i can’t log into the learn-on-demand?'
a1_a2 = 'i do not have time to complete my assignment.how many times can i upload the assignments?who will score my assignment?when can i see my assignment score?till which date i can complete assignments?what if i do not complete the assignments on the time? what if i do not attempt the assignments?where can i see my progress?what should i do if i did not score 70% for my assignment?who can i approach to reset my assignment?why i can’t log into the learn-on-demand?where do i upload the lab assignment?why can’t i see the upload button in the lms?why can’t i launch the lab?where do i get the template?what is the site to access my e-book?can i download or print the e-book?do you have a hard copy of the e-book?is it compulsory to attend the mentoring session?i am not able to attend the mentoring session.how was the attendance capture for the mentoring session?i have no issue with my assignment. can i skip the mentoring session?i have completed my assignment in advance. can i skip the mentoring session?what is the next step?how was the attendance capture for the mentoring session?why i can’t log into the learn-on-demand?'
a2_el3 = 'i am having an issue in accessing the e-content in lms.i am having difficulty in understanding the e-content.is it compulsory to complete the e-learning?till which date i can complete e-learning?what if i do not complete the e-learning on time? i do not have time to complete my mcqs.how many times can i attempt the mcqs?what should i do?till which date i can complete mcqs?what if i do not complete the mcqs on the time? what if i do not attempt the mcqs?is there is any marking for mcqs?where can i see my progress?i do not have time to complete my assignment.how many times can i upload the assignments?who will score my assignment?when can i see my assignment score?till which date i can complete assignments?what if i do not complete the assignments on the time? what if i do not attempt the assignments?what should i do if i did not score 70% for my assignment?who can i approach to reset my assignment?why i can’t log into the learn-on-demand?where do i upload the lab assignment?why can’t i see the upload button in the lms?why can’t i launch the lab?where do i get the template?what is the site to access my e-book?can i download or print the e-book?do you have a hard copy of the e-book?is it compulsory to attend the mentoring session?i am not able to attend the mentoring session.how was the attendance capture for the mentoring session?i have no issue with my assignment. can i skip the mentoring session?i have completed my assignment in advance. can i skip the mentoring session?what is the next step?how was the attendance capture for the mentoring session?'
el3_el4 = 'i am having an issue in accessing the e-content in lms.i am having difficulty in understanding the e-content.i am having difficulty in understanding the course content.is it compulsory to complete the e-learning?till which date i can complete e-learning?what if i do not complete the e-learning on time? i do not have time to complete my mcqs.how many times can i attempt the mcqs?what should i do?till which date i can complete mcqs?what if i do not complete the mcqs on the time? what if i do not attempt the mcqs?is there is any marking for mcqs?where can i see my progress?what is the site to access my e-book?can i download or print the e-book?do you have a hard copy of the e-book?what is the next step?'
el4_fc2 = 'i am having an issue in accessing the e-content in lms.i am having difficulty in understanding the e-content.i am having difficulty in understanding the course content.is it compulsory to complete the e-learning?till which date i can complete e-learning?what if i do not complete the e-learning on time? i do not have time to complete my mcqs.how many times can i attempt the mcqs?what should i do?till which date i can complete mcqs?what if i do not complete the mcqs on the time? what if i do not attempt the mcqs?is there is any marking for mcqs?where can i see my progress?when is my flipped class?what should i prepare for my flipped class?what if i cannot attend the flipped class. what can i do?is there any make-up flipped class?do i have to submit my assignment during flipped class?what is the site to access my e-book?can i download or print the e-book?do you have a hard copy of the e-book?what is the next step?do i have to submit my assignment during flipped class?'
fc2_a3 = 'i do not have time to complete my assignment.how many times can i upload the assignments?who will score my assignment?when can i see my assignment score?till which date i can complete assignments?what if i do not complete the assignments on the time? what if i do not attempt the assignments?where can i see my progress?what should i do if i did not score 70% for my assignment?who can i approach to reset my assignment?why i can’t log into the learn-on-demand?where do i upload the lab assignment?why can’t i see the upload button in the lms?why can’t i launch the lab?where do i get the template?what is the site to access my e-book?can i download or print the e-book?do you have a hard copy of the e-book?is it compulsory to attend the mentoring session?i am not able to attend the mentoring session.how was the attendance capture for the mentoring session?i have no issue with my assignment. can i skip the mentoring session?i have completed my assignment in advance. can i skip the mentoring session?what is the next step?when is my next flipped class?what should i prepare for my next flipped class?what if i cannot attend the next flipped class. what can i do?is there any make-up flipped class?do i have to submit my assignment during flipped class?'
a3_fc3 = 'i do not have time to complete my assignment.how many times can i upload the assignments?who will score my assignment?when can i see my assignment score?till which date i can complete assignments?what if i do not complete the assignments on the time? what if i do not attempt the assignments?where can i see my progress?what should i do if i did not score 70% for my assignment?who can i approach to reset my assignment?why i can’t log into the learn-on-demand?where do i upload the lab assignment?why can’t i see the upload button in the lms?why can’t i launch the lab?where do i get the template?what is the site to access my e-book?can i download or print the e-book?do you have a hard copy of the e-book?is it compulsory to attend the mentoring session?i am not able to attend the mentoring session.how was the attendance capture for the mentoring session?i have completed my assignment in advance. can i skip the mentoring session?what is the next step?i have no issue with my assignment. can i skip the mentoring session?'
fc3_el5 = 'i am having an issue in accessing the e-content in lms.i am having difficulty in understanding the e-content.i am having difficulty in understanding the course content.is it compulsory to complete the e-learning?till which date i can complete e-learning?what if i do not complete the e-learning on time? i do not have time to complete my mcqs.how many times can i attempt the mcqs?what should i do?till which date i can complete mcqs?what if i do not complete the mcqs on the time? what if i do not attempt the mcqs?is there is any marking for mcqs?where can i see my progress?i do not have time to complete my assignment.how many times can i upload the assignments?who will score my assignment?when can i see my assignment score?till which date i can complete assignments?what if i do not complete the assignments on the time? what if i do not attempt the assignments?what should i do if i did not score 70% for my assignment?who can i approach to reset my assignment?why i can’t log into the learn-on-demand?where do i upload the lab assignment?why can’t i see the upload button in the lms?why can’t i launch the lab?where do i get the template?what is the site to access my e-book?can i download or print the e-book?do you have a hard copy of the e-book?is it compulsory to attend the mentoring session?i am not able to attend the mentoring session.how was the attendance capture for the mentoring session?i have no issue with my assignment. can i skip the mentoring session?i have completed my assignment in advance. can i skip the mentoring session?what is the next step?'
el5_fc4 = 'i am having an issue in accessing the e-content in lms.i am having difficulty in understanding the e-content.i am having difficulty in understanding the course content.is it compulsory to complete the e-learning?till which date i can complete e-learning?what if i do not complete the e-learning on time? i do not have time to complete my mcqs.how many times can i attempt the mcqs?what should i do?till which date i can complete mcqs?what if i do not complete the mcqs on the time? what if i do not attempt the mcqs?is there is any marking for mcqs?where can i see my progress?what is the site to access my e-book?can i download or print the e-book?do you have a hard copy of the e-book?what is the next step?do you have a hard copy of the e-book?'
fc4_a4 = 'i am having an issue in accessing the e-content in lms.i am having difficulty in understanding the e-content.i am having difficulty in understanding the course content.is it compulsory to complete the e-learning?till which date i can complete e-learning?what if i do not complete the e-learning on time? i do not have time to complete my mcqs.how many times can i attempt the mcqs?what should i do?till which date i can complete mcqs?what if i do not complete the mcqs on the time? what if i do not attempt the mcqs?is there is any marking for mcqs?where can i see my progress?when is my flipped class?what should i prepare for my flipped class?what if i cannot attend the flipped class. what can i do?is there any make-up flipped class?do i have to submit my assignment during flipped class?what is the site to access my e-book?can i download or print the e-book?do you have a hard copy of the e-book?what is the next step?do i have to submit my assignment during flipped class?'
a4_a5 = 'i do not have time to complete my assignment.how many times can i upload the assignments?who will score my assignment?when can i see my assignment score?till which date i can complete assignments?what if i do not complete the assignments on the time? what if i do not attempt the assignments?where can i see my progress?what should i do if i did not score 70% for my assignment?who can i approach to reset my assignment?why i can’t log into the learn-on-demand?where do i upload the lab assignment?why can’t i see the upload button in the lms?why can’t i launch the lab?where do i get the template?what is the site to access my e-book?can i download or print the e-book?do you have a hard copy of the e-book?is it compulsory to attend the mentoring session?i am not able to attend the mentoring session.how was the attendance capture for the mentoring session?i have no issue with my assignment. can i skip the mentoring session?i have completed my assignment in advance. can i skip the mentoring session?what is the next step?when is my next flipped class?what should i prepare for my next flipped class?what if i cannot attend the next flipped class. what can i do?is there any make-up flipped class?do i have to submit my assignment during flipped class?'
a5_pm1 = 'i do not have time to complete my assignment.how many times can i upload the assignments?who will score my assignment?when can i see my assignment score?till which date i can complete assignments?what if i do not complete the assignments on the time? what if i do not attempt the assignments?where can i see my progress?what should i do if i did not score 70% for my assignment?who can i approach to reset my assignment?why i can’t log into the learn-on-demand?where do i upload the lab assignment?why can’t i see the upload button in the lms?why can’t i launch the lab?where do i get the template?what is the site to access my e-book?can i download or print the e-book?do you have a hard copy of the e-book?is it compulsory to attend the mentoring session?i am not able to attend the mentoring session.how was the attendance capture for the mentoring session?i have no issue with my assignment. can i skip the mentoring session?i have completed my assignment in advance. can i skip the mentoring session?what is the next step?when is my next flipped class?what should i prepare for my next flipped class?what if i cannot attend the next flipped class. what can i do?is there any make-up flipped class?do i have to submit my assignment during flipped class?i have no issue with my assignment. can i skip the mentoring session?'
pm1_pi1 = 'i do not have time to complete my assignment.how many times can i upload the assignments?who will score my assignment?when can i see my assignment score?till which date i can complete assignments?what if i do not complete the assignments on the time?what if i do not attempt the assignments?where can i see my progress?what should i do if i did not score 70% for my assignment?who can i approach to reset my assignment?why i can’t log into the learn-on-demand?where do i upload the lab assignment?why can’t i see the upload button in the lms?why can’t i launch the lab?where do i get the template?what is the site to access my e-book?can i download or print the e-book?do you have a hard copy of the e-book?is it compulsory to attend the mentoring session?i am not able to attend the mentoring session.how was the attendance capture for the mentoring session?i have no issue with my assignment. can i skip the mentoring session?i have completed my assignment in advance. can i skip the mentoring session?what is the next step?when is my next flipped class?what should i prepare for my next flipped class?what if i cannot attend the next flipped class. what can i do?is there any make-up flipped class?do i have to submit my assignment during flipped class?do you have a hard copy of the e-book?'

In [7]:
# Irritating nltk.sent_tokenizer recognizes sentences by .!? and space behind
# So need to add spaces
soc_days = soc_days.replace('?', '? ').replace('!', '! ').replace('.', '. ')
data_lower = soc_days

In [8]:


# Converting to word & sentence tokens
sent_token = nltk.sent_tokenize(data_lower)
word_token = nltk.word_tokenize(data_lower)

In [9]:
lemmer = nltk.stem.WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [10]:
greet_input = ['hello', 'hi', 'greetings', 'sup',
               "what's up", 'hey', 'yo', 'henlo']
greet_reply = ['Hello nigga', 'Hi homie', 'Yo wassup', 'Ni hao',
               'Konichiwa', 'Annyeong haseyo', 'Sawadikap']

def greeting(sentence): 
    for word in sentence.split():
        if word.lower() in greet_input:
            return random.choice(greet_reply)

In [11]:
def response(user_input):
    global index, flat, index
    bot_response = ''
    sent_token.append(user_input)
    
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopwords, ngram_range=(1,2))
#    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopwords, ngram_range=(1,2))
    tfidf = TfidfVec.fit_transform(sent_token)
    cosine_vals = cosine_similarity(tfidf[-1], tfidf)
#    cvec = CountVectorizer(tokenizer=LemNormalize, 
#                           stop_words='english',
#                           ngram_range=(1,2))
#    cvector = cvec.fit_transform(sent_token)
#    cosine_vals = cosine_similarity(cvector[-1], cvector)
    
    # Use to index out the matched response after sorting it
    # [-2] means 2nd from behind, use -2 because -1 is the input sentence
    # So, -2 is the next best score for cosine similarity
    index = cosine_vals.argsort()[0][-2]
    
    flat = cosine_vals.flatten()
    flat.sort()
    
    matched_cosine = flat[-2]
    
    if matched_cosine == 0:
        bot_response = bot_response + "I'm sorry, I do not understand you."
    
    else:
        try:
            bot_response = bot_response + qandadb[(qandadb['questions'].str.contains(sent_token[index]))]['resp'][qandadb[qandadb['questions'] == sent_token[index]]['resp'].index[0]]
        except:
            bot_response = bot_response + qandadb[(qandadb['questions'].str.contains(sent_token[index]))]
        # library[sent_token[index]]
        return bot_response

In [ ]:
flag=True
print("Hello, please ask me a question!")
while(flag==True):
    user_input = input()
    user_input = user_input.lower()
    
    if greeting(user_input) != None:
        print("Bot: " + greeting(user_input))
    
    elif user_input == 'thanks' or user_input == 'thank you' or user_input == 'bye':
        flag=False
        print("Bot: Bye! take care..")
    
    elif user_input != 'bye' or user_input != 'thanks' or user_input != 'thank you':
        print("Bot: ", end= "")
        print(response(user_input))
#        print(qandadb[(qandadb['questions'].str.contains(sent_token[index]))]['resp'])
        print('Cosine Similarity Values: ', flat)
        print('Matched Question: ', sent_token[index])
        print('2nd Best Match: ', sent_token[index-1])
        print('3rd Best Match:', sent_token[index-2])
        print(index)
        sent_token.remove(user_input)

Hello, please ask me a question!
pay with skills future?
Bot: 

C:\Users\kengw\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arent', 'couldnt', 'didnt', 'doe', 'doesnt', 'dont', 'ha', 'hadnt', 'hasnt', 'havent', 'isnt', 'mightnt', 'mustnt', 'neednt', 'shant', 'shes', 'shouldnt', 'shouldve', 'thatll', 'wa', 'wasnt', 'werent', 'wont', 'wouldnt', 'youd', 'youll', 'youre', 'youve'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


For payment via SFC, you are to place you claim via SkillFuture Credit website using your NRIC and your SingPass. Link: www.myskillsfuture.sg. A copy of the module invoice is required. You may request your LA for the invoice.
Cosine Similarity Values:  [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.21249242 0.8081374  1.        ]
Matched Question:  how do i pay with my skills future credits?
2nd Best Match:  i want to meet with my consultant.
3rd Best Match: can i request for an extension of payment?
7
No, i don't need help
Bot: 

C:\Users\kengw\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arent', 'couldnt', 'didnt', 'doe', 'doesnt', 'dont', 'ha', 'hadnt', 'hasnt', 'havent', 'isnt', 'mightnt', 'mustnt', 'neednt', 'shant', 'shes', 'shouldnt', 'shouldve', 'thatll', 'wa', 'wasnt', 'werent', 'wont', 'wouldnt', 'youd', 'youll', 'youre', 'youve'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Awesome! I’ll be here if you need any assistance. Otherwise, Happy Learning!
Cosine Similarity Values:  [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.13945344 0.51804888 0.69091913 1.        ]
Matched Question:  no, i do not need help.
2nd Best Match:  when must i submit my sfc?
3rd Best Match: how do i pay with my sfc?
10
i need help
Bot: 

...

How can we help you?
Cosine Similarity Values:  [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.13528034 0.5157619  0.77880697 1.         1.        ]
Matched Question:  yes, i need help.
2nd Best Match:  no, i do not need help.
3rd Best Match: when must i submit my sfc?
11
How can i contact my mentor online?
Bot: 

C:\Users\kengw\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arent', 'couldnt', 'didnt', 'doe', 'doesnt', 'dont', 'ha', 'hadnt', 'hasnt', 'havent', 'isnt', 'mightnt', 'mustnt', 'neednt', 'shant', 'shes', 'shouldnt', 'shouldve', 'thatll', 'wa', 'wasnt', 'werent', 'wont', 'wouldnt', 'youd', 'youll', 'youre', 'youve'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


You can contact your mentor through email between 9 am to 7pm.
Cosine Similarity Values:  [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.097899
 0.10827378 0.10827378 0.72131137 0.78704746 1.        ]
Matched Question:  how do i contact my mentor for online support?
2nd Best Match:  when can i contact my mentor?
3rd Best Match: how does the attendance work?
19
What time can contact mentor?
Bot: 

C:\Users\kengw\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arent', 'couldnt', 'didnt', 'doe', 'doesnt', 'dont', 'ha', 'hadnt', 'hasnt', 'havent', 'isnt', 'mightnt', 'mustnt', 'neednt', 'shant', 'shes', 'shouldnt', 'shouldve', 'thatll', 'wa', 'wasnt', 'werent', 'wont', 'wouldnt', 'youd', 'youll', 'youre', 'youve'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


You can contact your mentor through email between 9 am to 7pm.
Cosine Similarity Values:  [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.09190706
 0.10164685 0.10164685 0.36933636 0.67716326 1.        ]
Matched Question:  when can i contact my mentor?
2nd Best Match:  how does the attendance work?
3rd Best Match: how do we take attendance?
18


In [ ]:
LemNormalize('Yes, I have made my payment with my Consultant.')

In [ ]:
LemNormalize('How to make payment?')